In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

from buttermilk import BM
from promptflow.tracing import trace, start_trace
from rich import print as rprint

import os

from hydra import initialize, compose
from omegaconf import OmegaConf

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="./conf"):
    cfg = compose(config_name='config',
                  overrides=["+data=trans", "+step=trans", "+save=bq", "job=score","source=notebook"])

# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)

# Print config
rprint(OmegaConf.to_container(bm.cfg, resolve=True))



/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-11-06 11:15:29 J5HW6L4KT6 buttermilk buttermilk.py[ 297] INFO {'message': "Logging setup for: 20241106T0115Z-M8Tk-J5HW6L4KT6-suzor. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'automod', 'job': 'score', 'task_id': '20241106T0115Z-M8Tk-J5HW6L4KT6-suzor'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/automod/score/20241106T0115Z-M8Tk-J5HW6L4KT6-suzor", 'project': 'automod', 'job': 'score', 'run_id': '20241106T0115Z-M8Tk-J5HW6L4KT6-suzor', 'save_dir': 'gs://dmrc-analysis/runs/automod/score/20241106T0115Z-M8Tk-J5HW6L4KT6-suzor', 'ip': '159.196.210.27', 'node_name': 'J5HW6L4KT6', 'username': 'suzor'}
2024-11-06 11:15:29 J5HW6L4KT6 buttermilk buttermilk.py[ 305] DEBUG Buttermilk version is: 0.2.0
Starting prompt flow service...


I0000 00:00:1730855729.651663 28160940 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


Start prompt flow service on 127.0.0.1:23333, version: 1.16.1.
You can stop the prompt flow service with the following command:'pf service stop'.



{
    'name': 'automod',
    'job': 'score',
    'source': 'notebook',
    'secret_provider': 'azure',
    'logger': 'gcp',
    'tracing': True,
    'verbose': True,
    'models_secret': 'models',
    'save_dest': 'gcp',
    'save_dir': None,
    'gcp': {'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'},
    'azure': {'vault': 'https://suzorvault.vault.azure.net/', 'resource_group': 'rg-suzor_ai'},
    'aws': {'secret_name': 'secret', 'region': 'us-east-1'},
    'step': [
        {
            'flow': 'judger',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc',
                'template': 'judge',
                'criteria': [
                    'trans_cte',
                    'trans_tja',
                    'trans_glaad',
                    'trans_hrc',
                    'trans_simplified',
                    'trans_factored'
                ],
                'formatting': 'json_rules',
                'model': ['haiku', 'llama31_70b', 'gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'record_id': 'record_id', 'content': 'content', 'title': 'title'},
            'data': None
        },
        {
            'flow': 'synth',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc_judge',
                'template': 'synthesise',
                'criteria': ['trans_simplified'],
                'formatting': 'json_rules',
                'model': ['haiku', 'gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'answers': 'answers', 'meta': 'meta'},
            'data': {
                'judger': {
                    'name': 'previous runs',
                    'type': 'job',
                    'dataset': 'dmrc-analysis.toxicity.flow',
                    'max_records_per_group': 1,
                    'filter': {'agent_info.flow': 'judger'},
                    'group': {
                        'record_id': 'record.record_id',
                        'flow': 'agent_info.flow',
                        'template': 'agent_info.template',
                        'criteria': 'agent_info.template_vars.criteria',
                        'model': 'parameters.model'
                    },
                    'columns': {
                        'answers': {
                            'id': 'job_id',
                            'model': 'model',
                            'criteria': 'criteria',
                            'template': 'template',
                            'flow': 'flow',
                            'reasons': 'outputs.reasons'
                        },
                        'meta': {
                            'flow': 'agent_info.flow',
                            'template': 'agent_info.template',
                            'criteria': 'agent_info.template_vars.criteria',
                            'model': 'parameters.model',
                            'prediction': 'outputs.prediction',
                            'labels': 'outputs.labels',
                            'confidence': 'outputs.confidence',
                            'timestamp': 'timestamp'
                        }
                    }
                }
            }
        },
        {
            'flow': 'eval',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc',
                'template': 'differences',
                'criteria': ['trans_simplified'],
                'model': ['haiku', 'gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'answers': 'answers', 'meta': 'meta'},
            'data': {
                'judger': {
                    'name': 'previous runs and synth',
                    'type': 'job',
                    'dataset': 'dmrc-analysis.toxicity.flow',
                    'max_records_per_group': 1,
                    'filter': {'agent_

## First step assesses content against our criteria

In [3]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[0], data=cfg.data, save=cfg.save, source=cfg.source)


In [ ]:
await orchestrator.run_tasks()


2024-11-06 11:15:48 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_Y63Jb8 with job cLWruB8mzRSZNGYQ7mHPPV.


DEBUG:buttermilk:Starting task for Agent lc_Y63Jb8 with job cLWruB8mzRSZNGYQ7mHPPV.


2024-11-06 11:17:00 J5HW6L4KT6 buttermilk lc.py[ 204] DEBUG Invoking chain with haiku...


DEBUG:buttermilk:Invoking chain with haiku...


2024-11-06 11:17:01 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_Y63Jb8 with job XSG8q7NYvpXe6Egjd6J538.


...

DEBUG:buttermilk:Starting task for Agent lc_hyJL3P with job DushFYBE2asAkCZD4Hsux9.


2024-11-06 11:20:13 J5HW6L4KT6 buttermilk lc.py[ 204] DEBUG Invoking chain with gemini15pro...


DEBUG:buttermilk:Invoking chain with gemini15pro...
I0000 00:00:1730856019.541340 28160940 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


2024-11-06 11:20:21 J5HW6L4KT6 buttermilk lc.py[ 215] DEBUG Invoked chain with gpt4o in 65.75 seconds


DEBUG:buttermilk:Invoked chain with gpt4o in 65.75 seconds
I0000 00:00:1730856021.740277 28160940 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1730856022.169509 28160940 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


## Second step is a synthesis of previous draft answers

In [ ]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[1], data=cfg.data, save=cfg.save, source=cfg.source)


In [ ]:
# Inspect the dataset so we can make sure the inputs look ok
rprint(orchestrator._dataset.sample().answers.values)


In [ ]:
await orchestrator.run_tasks()


## Third step, we summarise the answers, similarities, and differences

In [ ]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[2], data=cfg.data, save=cfg.save, source=cfg.source)


In [ ]:
await orchestrator.run_tasks()
